In [1]:
import requests
from bs4 import BeautifulSoup
from csv import writer
from time import sleep
import pandas as pd

In [2]:
class TurboScrap:
    def pageSoup(self, url):
        response=requests.get(url)
        page=BeautifulSoup(response.text,"html.parser")
        return page
    
    def carUrls(self, page):
        urls_page=page.find_all(class_="products")[2].findAll('a')
        car_urls=[]
        for i in range(0, len( urls_page ), 3 ):
            car_urls.append(urls_page[i].get('href'))
        return car_urls
    
    def carInf(self,link):
        turbo_az="https://turbo.az"
        car_page=self.pageSoup( f"{turbo_az}{link}" )
        sleep(2)        
        car_info={
            'url': f"{turbo_az}{link}" 
        }
        car_info.update(self.carContactInfo(car_page))
        car_info.update(self.carProperty(car_page))
        car_info.update(self.carExtras(car_page))

        return car_info
    
    def carContactInfo(self, car_page):
        car_info = dict()
        
        try:
            car_info["Ad"] = car_page.find(class_="seller-info").find('p').text
            car_info["Telefon"] = car_page.find(class_="seller-info").find('a').text
        except:
            salon_page=car_page.find(class_="shop-header")
            car_info["Avtosalon adı"]=salon_page.find(class_='shop--title').text
            car_info["Avtosalon haqqında"]=salon_page.find(class_='shop--description').text
            car_info["Avtosalon ünvanı"]=salon_page.find(class_='shop--location-wrap').text
            
            phones=[]
            for phone in salon_page.find_all(class_='shop-phones-i'):
                phones.append(phone.text)
            car_info["Avtosalon telefonları"]=phones
            car_info["Avtosalon iş saatları"]=salon_page.find(class_='shop--schedule').text
        return car_info
    
    def carProperty(self, car_page):
        car_info = dict()
        
        properties_page=car_page.find_all("li", {"class":"product-properties-i"})
        for car_property in properties_page :
            try:
                key=car_property.label.text
                value=car_property.div.text
                car_info[key]=value
            except:
                continue
        return car_info
    
    def carExtras(self, car_page):
        car_info = dict()
        
        for tag in car_page.find(class_="product-statistics").find_all('p'):
            car_info[tag.text.split(':')[0]]=tag.text.split(':')[1]
            
        try:
            car_info["Haqqında"]=car_page.find(class_='product-text').p.text
        except:
            car_info["Haqqında"]=''
            
        extras= []
        try:
            for extra in car_page.find(class_="product-extras").find_all('p'):
                extras.append( extra.text )
        except:
            car_info['Extras']=extras
        car_info['Extras']=extras
        
        return car_info
    
    def runScrap(self, number):
        base="https://turbo.az/autos?page="
        cars=[]
        for page_number in range(number):
            response = self.pageSoup(f"{base}{page_number}")
            urls = self.carUrls(response)
            
            for link in urls:
                print(link)
                cars.append(self.carInf(link))
        return cars
    
    def test(self, number):
        base="https://turbo.az/autos?page="
        for link in self.carUrls( self.pageSoup(f"{base}{number}") ):
            print(link)

In [3]:
data=TurboScrap().runScrap(1)

/autos/4779766-opel-astra
/autos/4763516-skoda-fabia
/autos/4827578-lada-vaz-2104
/autos/4777306-bmw-320
/autos/4714863-kamaz-6520
/autos/4785982-iran-khodro-azsamand
/autos/4805542-mercedes-e-240
/autos/4695118-toyota-prado
/autos/4735932-jeep-grand-cherokee
/autos/4827576-hyundai-sonata
/autos/4758770-toyota-prado
/autos/4805534-mitsubishi-pajero
/autos/4770715-jeep-wrangler
/autos/4539390-ford-mustang
/autos/4777367-hyundai-i30
/autos/4759046-mitsubishi-asx
/autos/4817408-mercedes-c-180
/autos/4625888-lexus-lx-570
/autos/4701789-mercedes-e-200
/autos/4827573-kia-cerato
/autos/4827575-mercedes-e-270
/autos/4827574-bmw-528
/autos/4827571-mercedes-e-200
/autos/4827568-volkswagen-passat


In [4]:
df=pd.DataFrame(data)
df

,url,Ad,Telefon,Şəhər,Marka,Model,Buraxılış ili,Ban növü,Rəng,Mühərrik,...,Baxışların sayı,Yeniləndi,Elanın nömrəsi,Haqqında,Extras,Avtosalon adı,Avtosalon haqqında,Avtosalon ünvanı,Avtosalon telefonları,Avtosalon iş saatları
0,https://turbo.az/autos/4779766-opel-astra,Samir,(050) 826-55-05,Sumqayıt,Opel,Astra,2008,Hetçbek / Liftbek,Gümüşü,1.7 L,...,1925,02 Mart 2021,4779766,Salam. Mən həkiməm. Telefon söndürülmüş olsa d...,"[Yüngül lehimli disklər, ABS, Yağış sensoru, K...",NaN,NaN,NaN,NaN,NaN
1,https://turbo.az/autos/4763516-skoda-fabia,NaN,NaN,Bakı,Skoda,Fabia,2013,Hetçbek / Liftbek,Ağ,1.2 L,...,17547,02 Mart 2021,4763516,Avtomobil ideal veziyyetdedir.Kosmetik ve mexa...,"[Yüngül lehimli disklər, ABS, Yağış sensoru, M...","Avtosalon ""Otokoç Azerbaijan"" – Ikinci əl",Sənədləşmə və təhvil-təslim müddəti – 1 saat. ...,"Bakı ş., Yasamal r., İ.Qutqaşınlı küç. 50","[(012) 497-54-55, (055) 455-02-31]",Bazar ertəsi – Cümə: 09:00–18:00
2,https://turbo.az/autos/4827578-lada-vaz-2104,Elməddin,(055) 291-42-55,Bakı,LADA (VAZ),2104,2003,Universal,Ağ,1.5 L,...,13,02 Mart 2021,4827578,Maşın gündəlik sürülən maşındı mator bez nomer...,[],NaN,NaN,NaN,NaN,NaN
3,https://turbo.az/autos/4777306-bmw-320,NaN,NaN,Bakı,BMW,320,2003,Sedan,Tünd qırmızı,2.2 L,...,6614,02 Mart 2021,4777306,KREDiT MÜMKÜNDÜR\nMaximum kredit muddeti 24-36...,"[Yüngül lehimli disklər, ABS, Lyuk, Mərkəzi qa...","Avtosalon ""1 IST Cars Baku""",1 IST Cars avtosalonu сavan və dinamik sahibka...,"Bakı ş., Yasamal r., Tblisi pr. (İST Petrol)","[(050) 209-10-12, (055) 209-10-12, (070) 209-1...",Hər gün: 09:00–21:00
4,https://turbo.az/autos/4714863-kamaz-6520,Atif,(055) 331-02-77,Bakı,KamAz,6520,1992,Yük maşını,Qırmızı,11 L,...,5165,02 Mart 2021,4714863,"Mator 1.1, Rama təmiz, resorlar dayevo, sənəd ...",[],NaN,NaN,NaN,NaN,NaN
5,https://turbo.az/autos/4785982-iran-khodro-azs...,Vahid,(050) 681-89-37,Bakı,Iran Khodro,Azsamand,2006,Sedan,Göy,1.8 L,...,5163,02 Mart 2021,4785982,Maşın həm nağd həmdə kredit yolu ilə satılır.\...,[],NaN,NaN,NaN,NaN,NaN
6,https://turbo.az/autos/4805542-mercedes-e-240,Valeh,(055) 378-76-97,Bakı,Mercedes,E 240,1998,Sedan,Göy,2.4 L,...,333,02 Mart 2021,4805542,Tecili satilir çox səliqəli maşındır vuruqu ud...,"[Yüngül lehimli disklər, ABS, Yağış sensoru, M...",NaN,NaN,NaN,NaN,NaN
7,https://turbo.az/autos/4695118-toyota-prado,NaN,NaN,Bakı,Toyota,Prado,2008,Offroader / SUV,Ağ,4.0 L,...,4527,02 Mart 2021,4695118,🛑🛑🛑2008 buraxilis-2009 alinib🛑🛑🛑\nTam ideal ve...,"[Yüngül lehimli disklər, ABS, Lyuk, Yağış sens...","Avtosalon ""444""",Avtosalonumuz 18 ildir fəaliyyət göstərir. Biz...,"Bakı ş., Xətai r., Babək pr., 2270/76","[(012) 424-32-47, (050) 322-02-15, (055) 322-0...",Hər gün: 08:00–20:00
8,https://turbo.az/autos/4735932-jeep-grand-cher...,NaN,NaN,Bakı,Jeep,Grand Cherokee,2016,Offroader / SUV,Qara,3.6 L,...,2762,02 Mart 2021,4735932,🛑🛑🛑EVROPEKA🛑🛑🛑\n🛑2016 buraxilis-2020.02 de Avt...,"[Yüngül lehimli disklər, ABS, Lyuk, Yağış sens...","Avtosalon ""444""",Avtosalonumuz 18 ildir fəaliyyət göstərir. Biz...,"Bakı ş., Xətai r., Babək pr., 2270/76","[(012) 424-32-47, (050) 322-02-15, (055) 322-0...",Hər gün: 08:00–20:00
9,https://turbo.az/autos/4827576-hyundai-sonata,Tural,(050) 723-06-26,Bakı,Hyundai,Sonata,2010,Sedan,Bənövşəyi,2.4 L,...,31,02 Mart 2021,4827576,DİQQƏTLƏ OXUYUN TƏCİLİ SATIRAM.İŞİMLƏ BAĞLI PU...,"[Yüngül lehimli disklər, ABS, Lyuk, Yağış sens...",NaN,NaN,NaN,NaN,NaN


In [15]:
def difference(url):
    page1=TurboScrap().pageSoup(url)
    urls1=TurboScrap().carUrls(page1)
#     print(urls1)
    sleep(5)
    page1=TurboScrap().pageSoup(url)
    urls2=TurboScrap().carUrls(page1)
#     print(urls2)
    return urls1, urls2

for i in range(30):
    list1, list2=difference('https://turbo.az/autos?page=1')
    list_difference = [item for item in list2 if item not in list1]
    sleep(2)
    print(f'attemp:{ i}    Difference: {len(list_difference)}')
    if len(list_difference)>0:
        for url in list_difference:
            print( TurboScrap.carInf(url) )

attemp:0    Difference: 0
attemp:1    Difference: 0
attemp:2    Difference: 0
attemp:3    Difference: 0
attemp:4    Difference: 0
attemp:5    Difference: 0
attemp:6    Difference: 0
attemp:7    Difference: 0
attemp:8    Difference: 0
attemp:9    Difference: 0
attemp:10    Difference: 0
attemp:11    Difference: 0
attemp:12    Difference: 0
attemp:13    Difference: 0
attemp:14    Difference: 0
attemp:15    Difference: 0
attemp:16    Difference: 0
attemp:17    Difference: 0
attemp:18    Difference: 1


TypeError: carInf() missing 1 required positional argument: 'link'

In [5]:
import psycopg2

In [34]:
conn = psycopg2.connect(database = "turbo", user = "postgres", password = "3129246", host = "127.0.0.1", port = "5432")

In [35]:
cur = conn.cursor()

In [34]:
cur.execute('''CREATE TABLE TURBO
      (ID INT PRIMARY KEY     NOT NULL,
      ad           TEXT    NOT NULL,
      AGE            INT     NOT NULL,
      ADDRESS        CHAR(50),
      SALARY         REAL);''')

In [126]:
test='sadiq'

In [129]:
cur.execute(f'ALTER TABLE COMPANIES ADD {test} VARCHAR(50);')

In [176]:
conn.commit()
conn.close()

In [290]:
conn = psycopg2.connect(database = "turbo", user = "postgres", password = "3129246", host = "127.0.0.1", port = "5432")

In [291]:
cur = conn.cursor()

In [256]:
conn.commit()
conn.close()

InterfaceError: connection already closed

In [236]:
df.columns

Index(['url', 'Ad', 'Telefon', 'Şəhər', 'Marka', 'Model', 'Buraxılış ili',
       'Ban növü', 'Rəng', 'Mühərrik', 'Mühərrikin gücü', 'Yanacaq növü',
       'Yürüş', 'Sürətlər qutusu', 'Ötürücü', 'Yeni', 'Qiymət',
       'Baxışların sayı', 'Yeniləndi', 'Elanın nömrəsi', 'Haqqında', 'Extras',
       'Avtosalon adı', 'Avtosalon haqqında', 'Avtosalon ünvanı',
       'Avtosalon telefonları', 'Avtosalon iş saatları'],
      dtype='object')

In [ ]:
data

In [240]:
for car in data:
    if len(car.keys())>22:
        print(car['url'])
#         print(car['url'])
#         print(car['url'])
#         print(car['url'])
#         print(car['url'])
#         print(car['url'])
#         print(car['url'])
#         print(car['url'])
    

https://turbo.az/autos/4823684-renault-megane
https://turbo.az/autos/4588794-peugeot-206
https://turbo.az/autos/4571242-volkswagen-touareg
https://turbo.az/autos/4588218-hyundai-santa-fe
https://turbo.az/autos/4777977-infiniti-fx35
https://turbo.az/autos/4742641-nissan-x-trail


In [297]:
cur.execute(f'''CREATE TABLE mtest3
      (ID INT PRIMARY KEY     NOT NULL,
      ad           TEXT    NOT NULL,
      telefon            CHAR(50)     NOT NULL,
      şəhər        CHAR(50),
      marka        CHAR(50),
      model        CHAR(50),
      buraxılış_ili        CHAR(50),
      ban_növü        CHAR(50),
      rəng        CHAR(50),
      mühərrik        CHAR(50),
      güc        CHAR(50),
      yanacaq_növü        CHAR(50),
      yürüş        CHAR(50),
      sürətlər_qutusu        CHAR(50),
      ötürücü        CHAR(50),
      yeni        CHAR(50),
      qiymət        CHAR(50),
      baxış_sayı        CHAR(50),
      yeniləndi        CHAR(50),
      haqqında        CHAR(50),
      extras        CHAR(50),
      url         CHAR(50));''')

In [163]:
data

[{'url': 'https://turbo.az/autos/4807361-hyundai-santa-fe',
  'Ad': 'Qosqar',
  'Telefon': '(070) 354-20-08',
  'Şəhər': 'Bakı',
  'Marka': 'Hyundai',
  'Model': 'Santa Fe',
  'Buraxılış ili': '2012',
  'Ban növü': 'Offroader / SUV',
  'Rəng': 'Ağ',
  'Mühərrik': '2.4 L',
  'Mühərrikin gücü': '184 a.g.',
  'Yanacaq növü': 'Benzin',
  'Yürüş': '91 815 km',
  'Sürətlər qutusu': 'Avtomat',
  'Ötürücü': 'Tam',
  'Yeni': 'Xeyr',
  'Qiymət': '32 000 AZN',
  'Baxışların sayı': ' 1',
  'Yeniləndi': ' 23 Fevral 2021',
  'Elanın nömrəsi': ' 4807361',
  'Haqqında': 'Əlavə məlumat üçün müraciət edin. ',
  'Extras': ['Yüngül lehimli disklər',
   'ABS',
   'Lyuk',
   'Yağış sensoru',
   'Mərkəzi qapanma',
   'Park radarı',
   'Kondisioner',
   'Oturacaqların isidilməsi',
   'Dəri salon',
   'Ksenon lampalar',
   'Arxa görüntü kamerası',
   'Yan pərdələr',
   'Oturacaqların ventilyasiyası']},
 {'url': 'https://turbo.az/autos/4807360-opel-astra',
  'Ad': 'Niyazi',
  'Telefon': '(055) 313-22-32',
  'Şə

In [344]:

conn = psycopg2.connect(database = "turbo", user = "postgres", password = "3129246", host = "127.0.0.1", port = "5432")
cur = conn.cursor()

In [346]:
conn.commit()
conn.close()

In [314]:
# cur.execute(f"INSERT INTO COMPANIES (ID,NAME,AGE,ADDRESS,SALARY) \
#       VALUES (1, 'Paul', 32, 'California', 20000.00 )");
for car in data:
    if len(car.keys())>22:
        continue
    
    cur.execute( 
        f"INSERT INTO mtest3 VALUES ( {car['Elanın nömrəsi']} , '{car['Ad']}', '{car['Telefon']}' , '{car['Şəhər']}', '{car['Marka']}', '{car['Model']}', '{car['Buraxılış ili']}', '{car['Ban növü']}', '{car['Rəng']}', '{car['Mühərrik']}', '{car['Mühərrikin gücü']}', '{car['Yanacaq növü']}', '{car['Yürüş']}' , '{car['Sürətlər qutusu']}' , '{car['Ötürücü']}' , '{car['Yeni']}', '{car['Qiymət']}', '{car['Baxışların sayı']}', '{car['Yeniləndi']}', '{car['Haqqında']}', '{listToString(car['Extras'])}' , '{car['url']}' );" 
    )

In [38]:
def listToString(l):
    st=''
    for a in l:
        st+=' '+a
    return st
# listToString (data[1]['Extras'])
type(data[1]['Extras'])

list

In [327]:
cur.execute(f'''CREATE TABLE mtest3_avtosalon
      (ID INT PRIMARY KEY     NOT NULL,
      avtosalon_ad           TEXT    NOT NULL,
      avtosalon_haqqında     TEXT,
      avtosalon_ünvanı       TEXT,
      avtosalon_iş_saatları  TEXT,
      avtosalon_telefon     CHAR(50)     NOT NULL,
      şəhər        CHAR(50),
      marka        CHAR(50),
      model        CHAR(50),
      buraxılış_ili        CHAR(50),
      ban_növü        CHAR(50),
      rəng        CHAR(50),
      mühərrik        CHAR(50),
      güc        CHAR(50),
      yanacaq_növü        CHAR(50),
      yürüş        CHAR(50),
      sürətlər_qutusu        CHAR(50),
      ötürücü        CHAR(50),
      yeni        CHAR(50),
      qiymət        CHAR(50),
      baxış_sayı        CHAR(50),
      yeniləndi        CHAR(50),
      haqqında        TEXT,
      extras        TEXT,
      url         TEXT);''')

In [345]:
# cur.execute(f"INSERT INTO COMPANIES (ID,NAME,AGE,ADDRESS,SALARY) \
#       VALUES (1, 'Paul', 32, 'California', 20000.00 )");
for car in data:
    if len(car.keys())>22:
#         print(f"INSERT INTO mtest3_avtosalon VALUES ( {car['Elanın nömrəsi']} , '{car['Avtosalon adı']}', '{car['Avtosalon haqqında']}' , '{car['Avtosalon ünvanı']}','{car['Avtosalon iş saatları']}', '{listToString(car['Avtosalon telefonları'])}', '{car['Marka']}', '{car['Model']}', '{car['Buraxılış ili']}', '{car['Ban növü']}', '{car['Rəng']}', '{car['Mühərrik']}', '{car['Mühərrikin gücü']}', '{car['Yanacaq növü']}', '{car['Yürüş']}' , '{car['Sürətlər qutusu']}' , '{car['Ötürücü']}' , '{car['Yeni']}', '{car['Qiymət']}', '{car['Baxışların sayı']}', '{car['Yeniləndi']}', '{car['Haqqında']}', '{listToString(car['Extras'])}' , '{car['url']}' );")
        cur.execute( 
        f"INSERT INTO mtest3_avtosalon VALUES ( {car['Elanın nömrəsi']} , '{car['Avtosalon adı']}', '{car['Avtosalon haqqında']}' , '{car['Avtosalon ünvanı']}','{car['Avtosalon iş saatları']}', '{listToString(car['Avtosalon telefonları'])}','{car['Şəhər']}', '{car['Marka']}', '{car['Model']}', '{car['Buraxılış ili']}', '{car['Ban növü']}', '{car['Rəng']}', '{car['Mühərrik']}', '{car['Mühərrikin gücü']}', '{car['Yanacaq növü']}', '{car['Yürüş']}' , '{car['Sürətlər qutusu']}' , '{car['Ötürücü']}' , '{car['Yeni']}', '{car['Qiymət']}', '{car['Baxışların sayı']}', '{car['Yeniləndi']}', '{car['Haqqında']}', '{listToString(car['Extras'])}' , '{car['url']}' );" 
    )

In [15]:
sql = 'insert into table (a, b, c, d) values(%s, %s, %s, %s);'
values = (1,)

In [18]:
return a, b

In [23]:
a = [1, 2, 3]

In [20]:
a, b = my_function()

NameError: name 'my_function' is not defined

In [54]:
conn = psycopg2.connect(database = "turbo", user = "postgres", password = "3129246", host = "127.0.0.1", port = "5432")
cur = conn.cursor()

In [57]:
conn.commit()
conn.close()

In [6]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

In [70]:
driver = webdriver.Firefox()

driver.get('https://turbo.az/autos/new')
driver.find_element_by_name("phone[1][number]").send_keys("(051) 438-39-40")
driver.find_element_by_xpath("/html/body/div[2]/div[3]/div[2]/div[1]/div[1]/div[3]/div[3]").click()

html=driver.page_source
soup=BeautifulSoup(html)

markalar_page=soup.find('select', {'name':'auto[make_id]'}).findAll('option')
markalar_page
markalar=dict()
for marka in list(markalar_page[1:]):
    markalar[marka.text]=marka['value']
    
ban_page=soup.find('select', {'name':'auto[category_id]'}).findAll('option')
bans=[]
for ban in ban_page[1:]:
    bans.append(ban.text)
    
color_page=soup.find('select', {'name':'auto[color_id]'}).findAll('option')
colors=[]
for color in color_page[1:]:
    colors.append(color.text)

fuel_page=soup.find('select', {'name':'auto[fuel_type_id]'}).findAll('option')
fuels=[]
for fuel in fuel_page[1:]:
    fuels.append(fuel.text)

gear_page=soup.find('select', {'name':'auto[gear_id]'}).findAll('option')
gears=[]
for gear in gear_page[1:]:
    gears.append(gear.text)

transmission_page=soup.find('select', {'name':'auto[transmission_id]'}).findAll('option')
transmissions=[]
for transmission in transmission_page[1:]:
    transmissions.append(transmission.text)

region_page=soup.find('select', {'name':'auto[region_id]'}).findAll('option')
regions=[]
for region in region_page[1:]:
    regions.append(region.text)

    

select=Select(driver.find_element_by_name("auto[make_id]"))

for marka in markalar:
    select.select_by_value(markalar[marka])
    model_soup=BeautifulSoup(driver.page_source)
    modeller_page=model_soup.find('select', {'name':'auto[model_id]'}).findAll('option')
    models=[]
    for model in modeller_page[1:]:
        models.append(model.text)
    markalar[marka]=models
    

driver.quit()

In [532]:
conn = psycopg2.connect(database = "turbo", user = "postgres", password = "3129246", host = "127.0.0.1", port = "5432")

In [533]:
cur = conn.cursor()

In [353]:
cur.execute(f'''CREATE TABLE REGION
      (id SERIAL PRIMARY KEY     NOT NULL,
       region           CHAR(50)    NOT NULL);''')

In [425]:
conn.commit()

In [426]:
conn.close()

In [362]:
for i in range(len(regions)):
    cur.execute(f"INSERT INTO REGION(region) VALUES ('{regions[i]}');")

In [354]:
cur.execute(f'''CREATE TABLE TRANSMISSION
      (id SERIAL PRIMARY KEY     NOT NULL,
       transmission           CHAR(50)    NOT NULL);''')

In [363]:
for i in range(len(transmissions)):
    cur.execute(f"INSERT INTO TRANSMISSION(transmission) VALUES ('{transmissions[i]}');")

In [355]:
cur.execute(f'''CREATE TABLE GEAR
      (id SERIAL PRIMARY KEY     NOT NULL,
       gear           CHAR(50)    NOT NULL);''')

In [364]:
for i in range(len(gears)):
    cur.execute(f"INSERT INTO GEAR(gear) VALUES ('{gears[i]}');")

In [356]:
cur.execute(f'''CREATE TABLE FUEL
      (id SERIAL PRIMARY KEY     NOT NULL,
       fuel           CHAR(50)    NOT NULL);''')

cur.execute(f'''CREATE TABLE COLOR
      (id SERIAL PRIMARY KEY     NOT NULL,
       color           CHAR(50)    NOT NULL);''')

In [346]:
cur.execute(f'''CREATE TABLE BAN
      (id SERIAL PRIMARY KEY     NOT NULL,
       ban           CHAR(50)    NOT NULL);''')

In [365]:
for i in range(len(fuels)):
    cur.execute(f"INSERT INTO FUEL(fuel) VALUES ('{fuels[i]}');")

for i in range(len(colors)):
    cur.execute(f"INSERT INTO COLOR(color) VALUES ('{colors[i]}');")

In [348]:
for i in range(len(bans)):
    cur.execute(f"INSERT INTO BAN(BAN) VALUES ('{bans[i]}');")

In [404]:
cur.execute(f'''CREATE TABLE MARKA
      (id SERIAL PRIMARY KEY     NOT NULL,
       marka           CHAR(50)    NOT NULL);''')

In [405]:
for marka in markalar:
    cur.execute(f"INSERT INTO MARKA VALUES (DEFAULT, '{marka}');")

In [410]:
cur.execute(f'''CREATE TABLE MODEL
      (id SERIAL PRIMARY KEY     NOT NULL,
       model CHAR(50)      NOT NULL,
       marka_id  INT    references MARKA(id)
       );''')

In [424]:
for marka in markalar:
    cur.execute(f"select id from marka where marka='{marka}'")
    marka_id=cur.fetchone()[0]
#     print(marka_id)
    for model in markalar[marka]:
        cur.execute(f"INSERT INTO MODEL VALUES (DEFAULT, '{model}',{marka_id});")

In [253]:
cur.execute(f'''CREATE TABLE avtosalon
      (ID INT PRIMARY KEY     NOT NULL,
      avtosalon_ad           TEXT    NOT NULL,
      avtosalon_haqqında     TEXT,
      avtosalon_ünvanı       TEXT,
      avtosalon_iş_saatları  TEXT,
      avtosalon_telefon     TEXT,
      region_id        INT,
      model_id        INT,
      buraxılış_ili        INT,
      ban_id        INT,
      color_id        INT,
      mühərrik        CHAR(50),
      güc        CHAR(50),
      fuel_id        INT,
      yürüş        CHAR(50),
      transmission_id        INT,
      gear_id        CHAR(50),
      yeni        CHAR(50),
      qiymət        CHAR(50),
      baxış_sayı        CHAR(50),
      yeniləndi        CHAR(50),
      haqqında        TEXT,
      extras        TEXT,
      url         TEXT);''')

In [256]:
driver = webdriver.Firefox()

driver.get('https://turbo.az/autos/new')
driver.find_element_by_name("phone[1][number]").send_keys("(051) 438-39-40")
driver.find_element_by_xpath("/html/body/div[2]/div[3]/div[2]/div[1]/div[1]/div[3]/div[3]").click()

html=driver.page_source
soup=BeautifulSoup(html)

driver.quit()

In [270]:
u_extras=[]
for u_extra in soup.find_all(class_='collection_check_boxes'):
    u_extras.append(u_extra.text)
u_extras

['Yüngül lehimli disklər',
 'ABS',
 'Lyuk',
 'Yağış sensoru',
 'Mərkəzi qapanma',
 'Park radarı',
 'Kondisioner',
 'Oturacaqların isidilməsi',
 'Dəri salon',
 'Ksenon lampalar',
 'Arxa görüntü kamerası',
 'Yan pərdələr',
 'Oturacaqların ventilyasiyası']

In [390]:
cur.execute(f'''CREATE TABLE EXTRAS
      (id SERIAL PRIMARY KEY     NOT NULL,
       extra           CHAR(50)    NOT NULL);''')

In [391]:
for i in range(len(u_extras)):
    cur.execute(f"INSERT INTO EXTRAS(extra) VALUES ('{u_extras[i]}');")

In [49]:
conn = psycopg2.connect(database = "turbo", user = "postgres", password = "3129246", host = "127.0.0.1", port = "5432")
cur = conn.cursor()

In [42]:
conn.commit()
conn.close()

In [288]:
df.columns

Index(['url', 'Ad', 'Telefon', 'Şəhər', 'Marka', 'Model', 'Buraxılış ili',
       'Ban növü', 'Rəng', 'Mühərrik', 'Mühərrikin gücü', 'Yanacaq növü',
       'Yürüş', 'Sürətlər qutusu', 'Ötürücü', 'Yeni', 'Qiymət',
       'Baxışların sayı', 'Yeniləndi', 'Elanın nömrəsi', 'Haqqında', 'Extras',
       'Avtosalon adı', 'Avtosalon haqqında', 'Avtosalon ünvanı',
       'Avtosalon telefonları', 'Avtosalon iş saatları'],
      dtype='object')

In [312]:
# for car in data:
#     print(car['Elanın nömrəsi'])
#     print(car['Extras'])

In [418]:
for car in data:
    print(car['Elanın nömrəsi'])
    for extra in car['Extras']:
        cur.execute(f"select * from extras where extra='{extra}'")
        extra_id = cur.fetchone()[0]
        
        cur.execute(f"INSERT INTO car_details VALUES(DEFAULT, {car['Elanın nömrəsi']}, {extra_id})")

In [397]:
cur.execute(f'''CREATE TABLE car_details
      (id SERIAL PRIMARY KEY     NOT NULL,
        car_id       INT,
        extras_id       INT references extras(id)
        );''')

In [534]:
cur.execute(f'''CREATE TABLE seller
      (ID SERIAL PRIMARY KEY     NOT NULL,
      tpye         CHAR(50),
      ad           TEXT,
      avtosalon_haqqında     TEXT,
      avtosalon_ünvanı       TEXT,
      avtosalon_iş_saatları  TEXT,
      telefon     TEXT
      );''')

In [535]:
cur.execute(f'''CREATE TABLE car
      (ID SERIAL PRIMARY KEY     NOT NULL,
      seller_id INT references seller(id),
      region_id        INT references region(id),
      model_id        INT references model(id),
      buraxılış_ili        INT,
      ban_id        INT references ban(id),
      color_id        INT references color(id),
      mühərrik        CHAR(50),
      güc        CHAR(50),
      fuel_id        INT references fuel(id),
      yürüş        CHAR(50),
      transmission_id        INT references transmission(id),
      gear_id        INT references gear(id),
      yeni        CHAR(50),
      qiymət        CHAR(50),
      baxış_sayı        CHAR(50),
      haqqında        TEXT,
      url         TEXT);''')

In [463]:
cur.execute(f'''CREATE TABLE car_specs
      (ID INT PRIMARY KEY     NOT NULL,
      );''')

In [56]:
data[4]

{'url': 'https://turbo.az/autos/4827578-lada-vaz-2104',
 'Ad': 'Elməddin',
 'Telefon': '(055) 291-42-55',
 'Şəhər': 'Bakı',
 'Marka': 'LADA (VAZ)',
 'Model': '2104',
 'Buraxılış ili': '2003',
 'Ban növü': 'Universal',
 'Rəng': 'Ağ',
 'Mühərrik': '1.5 L',
 'Mühərrikin gücü': '75 a.g.',
 'Yanacaq növü': 'Benzin',
 'Yürüş': '100 000 km',
 'Sürətlər qutusu': 'Mexaniki',
 'Ötürücü': 'Arxa',
 'Yeni': 'Xeyr',
 'Qiymət': '5 800 AZN',
 'Baxışların sayı': ' 13',
 'Yeniləndi': ' 02 Mart 2021',
 'Elanın nömrəsi': ' 4827578',
 'Haqqında': 'Maşın gündəlik sürülən maşındı mator bez nomer karopka 5 heç problem yoxdur peredok 2 həftə di yığılıb salon səliqəlidi ayaq altılar yeni alınıb sdenyalar yeni çexollanıb  mator 0307di',
 'Extras': []}

In [55]:
for car in data:
    if car.get('Ad'):
#         print('owner')
        cur.execute(f" select id from seller where ad='{car['Ad']}' and telefon='{car['Telefon']}' ")
        row=cur.fetchone()
        if row:
            seller_id=row[0]
#             print('seller was found with seller_id'+str(seller_id))
        else:
            seller_id=cur.execute(f"""INSERT INTO seller VALUES(
                   DEFAULT,
                   'owner',
                   '{car['Ad']}',
                   NULL,
                   NULL,
                   NULL,
                   '{car['Telefon']}'
                       ) returning id;""")
        
    else:
#         print('Avtosalon')
        cur.execute(f" select id from seller where ad='{car['Avtosalon adı']}' ")
        row=cur.fetchone()
        if row:            
            seller_id=row[0]
#            print('seller was found with id='+str(seller_id))
        else:
            seller_id=cur.execute(f"""INSERT INTO seller VALUES(
               DEFAULT,
               'avtosalon',
               '{car['Avtosalon adı']}',
               '{car['Avtosalon haqqında']}',
               '{car['Avtosalon ünvanı']}',
               '{car['Avtosalon iş saatları']}',
               '{listToString( car['Avtosalon telefonları'] ) }'
                   );""")
    
    cur.execute(f"select id from region where region='{car['Şəhər']}'")
    region_id=cur.fetchone()[0]
    
    cur.execute(f"select model.id from marka,model where marka.id=model.marka_id and marka.marka='{car['Marka']}' and model.model='{car['Model']}'")
    model_id=cur.fetchone()[0]
    
    cur.execute(f"select id from ban where ban='{car['Ban növü']}'")
    ban_id=cur.fetchone()[0]

    cur.execute(f"select id from color where color='{car['Rəng']}'")
    color_id=cur.fetchone()[0]

    cur.execute(f"select id from transmission where transmission='{car['Sürətlər qutusu']}'")
    transmission_id=cur.fetchone()[0]
    
    cur.execute(f"select id from gear where gear='{car['Ötürücü']}'")
    gear_id=cur.fetchone()[0]
    
    print(str(ban_id)+' '+str(color_id)+' '+str(transmission_id)+' '+str(gear_id))

4 7 1 2
4 1 1 2
15 1 1 1
14 15 2 1
17 12 1 1
14 6 1 2
14 6 2 1
10 1 2 3
10 10 2 3
14 3 2 2
10 10 2 3
10 10 2 3
10 16 2 3
6 10 2 2
4 1 2 2
10 15 2 3
14 7 2 1
10 10 2 3
14 16 2 1
14 16 2 2
14 7 2 1
14 10 2 1
14 7 2 1
14 7 2 2


In [54]:
data[3]

{'url': 'https://turbo.az/autos/4777306-bmw-320',
 'Avtosalon adı': 'Avtosalon "1 IST Cars Baku"',
 'Avtosalon haqqında': '1 IST Cars avtosalonu сavan və dinamik sahibkarlar tərəfindən idarə olunur. İşimizdə düzgünlük, halallıq və açıq rəqabət tərəfdarıyıq.',
 'Avtosalon ünvanı': 'Bakı ş., Yasamal r., Tblisi pr. (İST Petrol)',
 'Avtosalon telefonları': ['(050) 209-10-12',
  '(055) 209-10-12',
  '(070) 209-10-12'],
 'Avtosalon iş saatları': 'Hər gün: 09:00–21:00',
 'Şəhər': 'Bakı',
 'Marka': 'BMW',
 'Model': '320',
 'Buraxılış ili': '2003',
 'Ban növü': 'Sedan',
 'Rəng': 'Tünd qırmızı',
 'Mühərrik': '2.2 L',
 'Mühərrikin gücü': '170 a.g.',
 'Yanacaq növü': 'Benzin',
 'Yürüş': '257 413 km',
 'Sürətlər qutusu': 'Avtomat',
 'Ötürücü': 'Arxa',
 'Yeni': 'Xeyr',
 'Qiymət': '13 800 AZN',
 'Baxışların sayı': ' 6614',
 'Yeniləndi': ' 02 Mart 2021',
 'Elanın nömrəsi': ' 4777306',
 'Haqqında': 'KREDiT MÜMKÜNDÜR\nMaximum kredit muddeti 24-36 AY AZN-LE. Kreditle masin alan musterilerin nezerine: 2 s

In [32]:
# for car in data:
# if not car.get('Ad'):
cur.execute(f""" select * from seller where ad='Avtosalon "Start"' """)
if cur.fetchone():
    print('yes')

yes


In [622]:
cur.execute("select * from seller where ad='Sebuhi' and telefon='(070) 331-15-05'")
# cur.execute('select * from seller where id=2;')
a=cur.fetchone()
if not a:
    print('yes')

In [45]:
for car in data:
    
    if car.get('Ad'):
        print(car['Ad']+'  '+car['Telefon'])
    else:
        print(car['Avtosalon adı']+'  '+listToString(car['Avtosalon telefonları']))

Samir  (050) 826-55-05
Avtosalon "Otokoç Azerbaijan" – Ikinci əl   (012) 497-54-55 (055) 455-02-31
Elməddin  (055) 291-42-55
Avtosalon "1 IST Cars Baku"   (050) 209-10-12 (055) 209-10-12 (070) 209-10-12
Atif  (055) 331-02-77
Vahid  (050) 681-89-37
Valeh  (055) 378-76-97
Avtosalon "444"   (012) 424-32-47 (050) 322-02-15 (055) 322-02-15 (070) 322-02-15
Avtosalon "444"   (012) 424-32-47 (050) 322-02-15 (055) 322-02-15 (070) 322-02-15
Tural  (050) 723-06-26
Avtosalon "444"   (012) 424-32-47 (050) 322-02-15 (055) 322-02-15 (070) 322-02-15
Avtosalon "444"   (012) 424-32-47 (050) 322-02-15 (055) 322-02-15 (070) 322-02-15
Avtosalon "1 IST Cars Baku"   (050) 209-10-12 (055) 209-10-12 (070) 209-10-12
Avtosalon "1 IST Cars Baku"   (050) 209-10-12 (055) 209-10-12 (070) 209-10-12
Avtosalon "1 IST Cars Baku"   (050) 209-10-12 (055) 209-10-12 (070) 209-10-12
Avtosalon "1 IST Cars Baku"   (050) 209-10-12 (055) 209-10-12 (070) 209-10-12
Sərxan  (050) 572-56-76
Avtosalon "1 IST Cars Baku"   (050) 209-10

In [595]:
car_page=TurboScrap().pageSoup('https://turbo.az/autos/4823686-ford-fusion')

In [618]:
for a_link in car_page.find(class_="product-photos").findAll('a'):
    print(a_link.get('href'))

https://turbo.azstatic.com/uploads/full/2021%2F03%2F01%2F10%2F31%2F30%2F73d3fc6b-9fda-4284-addb-2fc56af493f2%2F75859_BQ2A5RZm8ckPY5IRLwrG0Q.jpg
https://turbo.azstatic.com/uploads/full/2021%2F03%2F01%2F10%2F31%2F43%2Fdfe4e9c5-3233-4e99-9ace-e6d9b844e380%2F69928_i3h7rLu7v6WFqAxPdcymHQ.jpg
https://turbo.azstatic.com/uploads/full/2021%2F03%2F01%2F10%2F31%2F58%2Fae7738b7-0c2c-4b7a-968c-30dc2f4e32c5%2F79869_SLZP5OSTVPB6wx76AVeKjw.jpg
https://turbo.azstatic.com/uploads/full/2021%2F03%2F01%2F10%2F32%2F13%2Fa63e8f68-f613-47cd-83ce-6d101a41aaa6%2F82458_h4z_XKh2V_O79c-TlxuhFA.jpg
https://turbo.azstatic.com/uploads/full/2021%2F03%2F01%2F10%2F32%2F29%2F29cfb9de-f4e2-4f08-a87b-f2da93723fe2%2F59870_kf9oLjs6ML5IjnxRoXtGfg.jpg
https://turbo.azstatic.com/uploads/full/2021%2F03%2F01%2F10%2F32%2F29%2F07c4952f-1fc9-4c9c-900c-8fd5c5414241%2F80671_miamS7hKQnNtZIKj_FoCrg.jpg
https://turbo.azstatic.com/uploads/full/2021%2F03%2F01%2F10%2F33%2F29%2F95f0fc0c-766d-4dbb-8937-7766568cd78d%2F69468_xoFrzjfpkvjqrdZ1zAI

In [599]:
import matplotlib.pyplot as plt
import matplotlib.image as mpim

In [40]:
data

[{'url': 'https://turbo.az/autos/4779766-opel-astra',
  'Ad': 'Samir',
  'Telefon': '(050) 826-55-05',
  'Şəhər': 'Sumqayıt',
  'Marka': 'Opel',
  'Model': 'Astra',
  'Buraxılış ili': '2008',
  'Ban növü': 'Hetçbek / Liftbek',
  'Rəng': 'Gümüşü',
  'Mühərrik': '1.7 L',
  'Mühərrikin gücü': '130 a.g.',
  'Yanacaq növü': 'Dizel',
  'Yürüş': '268 000 km',
  'Sürətlər qutusu': 'Mexaniki',
  'Ötürücü': 'Ön',
  'Yeni': 'Xeyr',
  'Qiymət': '10 900 AZN',
  'Baxışların sayı': ' 1925',
  'Yeniləndi': ' 02 Mart 2021',
  'Elanın nömrəsi': ' 4779766',
  'Haqqında': 'Salam. Mən həkiməm. Telefon söndürülmüş olsa deməli əməliyyatdayam. Xaiş edirəm az sonra təkrar zəng edəsiniz. ',
  'Extras': ['Yüngül lehimli disklər', 'ABS', 'Yağış sensoru', 'Kondisioner']},
 {'url': 'https://turbo.az/autos/4763516-skoda-fabia',
  'Avtosalon adı': 'Avtosalon "Otokoç Azerbaijan" – Ikinci əl',
  'Avtosalon haqqında': 'Sənədləşmə və təhvil-təslim müddəti – 1 saat. AVIS servisdə check-up və yağdəyişmə hədiyyə. 0 km-dən s